Importando librerías

In [ ]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from PIL import Image
from glob import glob
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import save_img
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D, LeakyReLU
import random
import matplotlib.pyplot as plt

#Se elije una carpeta desde donde se trabaja
mainpathlocation = '/media/fermin/Data/CosasUniversidad/Material/ULA6-Semestre12/Taller-de-Titulo-II/Recursos/HAM10000/'

In [ ]:
print(tf.__version__)
tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

#gpu_devices = tf.config.experimental.list_physical_devices('GPU')
#for device in gpu_devices:
#    tf.config.experimental.set_memory_growth(device, True)

Creando labels

In [ ]:
#Se lee el csv y se crean las labels de las clases
metadf = pd.read_csv(mainpathlocation+'HAM10000_metadata.csv')
le = LabelEncoder()
le.fit(metadf['dx'])
LabelEncoder()
print(list(le.classes_))

metadf['label'] = le.transform(metadf["dx"])
##print(metadf.sample(10))


#Actinic Keratoses (akiec)

#Basal Cell Carcinoma (bcc)

#Benign Keratosis-like (bkl)

#Dermatofibroma (df)

#Melanoma (mel)

#Vascular Lesions (vas)

#Melanocytic Nevi (nv)

Añadir las imágenes al dataframe

In [ ]:
os.chdir(mainpathlocation)
image_path = {os.path.splitext(os.path.basename(x))[0]: x
              for x in glob(os.path.join(mainpathlocation+'HAMTESTS/n6Resized/','*.jpg'))}
metadf['path'] = metadf['image_id'].map(image_path.get)

In [ ]:
os.chdir(mainpathlocation)
sizeX = 100
sizeY = 75
##Para normalizar y cambiar el tamaño con sizeX y sizeY
resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(sizeY, sizeX),
  layers.Rescaling(1./255)
])

metadf['image'] = metadf['path'].map(lambda x: np.asarray(resize_and_rescale(Image.open(x))))

Obtención de los datasets (por ahora solo el HAM10000)

In [ ]:
X = np.asarray(metadf['image'].tolist())
Y = metadf['label']
Y_cat = to_categorical(Y, num_classes=7)

x_train, x_test, y_train, y_test = train_test_split(X, Y_cat, test_size=0.2, random_state=42)

Creación del modelo

In [ ]:
model = tf.keras.Sequential()
model.add(Conv2D(256, (3, 3), activation="relu", input_shape=(sizeY, sizeX, 3)))
model.add(Conv2D(256, (3, 3)))
model.add(MaxPool2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(0.5))

model.add(Conv2D(128, (3, 3)))
model.add(Conv2D(128, (3, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3)))
model.add(Conv2D(64, (3, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(32, (3, 3)))
model.add(Conv2D(32, (3, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['acc'])

Entrenamiento

In [ ]:
batch_size = 16
epochs = 50

history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test,y_test), verbose=2)
model.save("navidadCompleja.keras")
np.save('navidadComplejaHistory.npy', history.history)

Cargar modelo

In [ ]:
#loaded_model = tf.keras.models.load_model("navidad.h5")
#history=np.load('navidadComplejaHistory.npy',allow_pickle='TRUE').item()

Evaluación de precisión

In [ ]:
test_eval = model.evaluate(x_test, y_test, verbose=1)
#test_eval = loaded_model.evaluate(x_test, y_test, verbose=1)
print('Acc:', test_eval[1])

Predicción de prueba

In [ ]:
predicciones = model.predict(x_test)
#predicciones = loaded_model.predict(x_test)

In [ ]:
seed = random.randrange(len(x_test))
print("La imágen ingresada es:")
_ = plt.imshow(x_test[seed])
print(np.argmax(y_test[seed]))
predicciones_classes = []
for prediction in predicciones:
    predicciones_classes.append(prediction.tolist().index(max(prediction)))
predicciones_classes = np.array(predicciones_classes)
print("La predicción es:")
print(predicciones_classes[seed])

In [ ]:
plt.xlabel('Epochs')
plt.ylabel("Magnitud de Pérdida")
plt.plot(history.history['loss'])

In [ ]:
accuracy = history.history['acc']
val_accuracy = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()